In [2]:
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    
    get_folders_by_plate_id,
)
from time import time_ns

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
directory_targ = os.path.join(directory_scratch,'stitch_temp')+'/'
plates = ['938_20220325']


In [3]:
directory_targ

'/scratch-shared/amftrack/stitch_temp/'

In [6]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher('dropbox_downloader.py',[directory_targ,name_job,0],plates,'20:00',name_job=name_job)

Submitted batch job 1142442


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [4]:
run_launcher('skeletonizer.py',[directory_targ],plates,'20:00',dependency=True)

Submitted batch job 1129540


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [5]:
run_launcher('skelet_video_maker.py',[directory_targ,name_job,0],plates,'20:00',dependency=True,name_job=name_job)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1142308


In [4]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

analysed:   0%|          | 0/71 [00:00<?, ?it/s]

In [7]:
all_folders['/Img/TileConfiguration.txt.registered']

0     True
1    False
2     True
Name: /Img/TileConfiguration.txt.registered, dtype: object

In [13]:
 i= 0
unique_ids = list(set(folders["unique_id"].values))
unique_ids.sort()
select = folders.loc[folders["unique_id"] == unique_ids[i]]

In [8]:
all_folders
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 14:35:",20220513_1435_Plate17,6_20220509,2022-05-13 14:35:00
1,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 19:35:",20220517_1935_Plate17,6_20220509,2022-05-17 19:35:00
2,/scratch-shared/amftrack/stitch_temp/20220511_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"11.05.2022, 19:50:",20220511_1950_Plate17,6_20220509,2022-05-11 19:50:00
3,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 06:56:",20220517_0656_Plate17,6_20220509,2022-05-17 06:56:00
4,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 10:35:",20220513_1035_Plate17,6_20220509,2022-05-13 10:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 04:32:",20220513_0432_Plate17,6_20220509,2022-05-13 04:32:00
67,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 17:35:",20220517_1735_Plate17,6_20220509,2022-05-17 17:35:00
68,/scratch-shared/amftrack/stitch_temp/20220520_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"20.05.2022, 15:37:",20220520_1537_Plate17,6_20220509,2022-05-20 15:37:00
69,/scratch-shared/amftrack/stitch_temp/20220520_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"20.05.2022, 13:37:",20220520_1337_Plate17,6_20220509,2022-05-20 13:37:00


In [10]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/skeleton_pruned_compressed.mat"]==True]
folders

analysed:   0%|          | 0/71 [00:00<?, ?it/s]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 14:35:",20220513_1435_Plate17,6_20220509,2022-05-13 14:35:00
3,/scratch-shared/amftrack/stitch_temp/20220517_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 06:56:",20220517_0656_Plate17,6_20220509,2022-05-17 06:56:00
4,/scratch-shared/amftrack/stitch_temp/20220513_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 10:35:",20220513_1035_Plate17,6_20220509,2022-05-13 10:35:00
7,/scratch-shared/amftrack/stitch_temp/20220514_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"14.05.2022, 06:33:",20220514_0633_Plate17,6_20220509,2022-05-14 06:33:00
9,/scratch-shared/amftrack/stitch_temp/20220512_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"12.05.2022, 13:50:",20220512_1350_Plate17,6_20220509,2022-05-12 13:50:00
11,/scratch-shared/amftrack/stitch_temp/20220512_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"12.05.2022, 20:32:",20220512_2032_Plate17,6_20220509,2022-05-12 20:32:00
13,/scratch-shared/amftrack/stitch_temp/20220512_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"12.05.2022, 22:32:",20220512_2232_Plate17,6_20220509,2022-05-12 22:32:00
14,/scratch-shared/amftrack/stitch_temp/20220516_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"16.05.2022, 02:37:",20220516_0237_Plate17,6_20220509,2022-05-16 02:37:00
16,/scratch-shared/amftrack/stitch_temp/20220518_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"18.05.2022, 13:48:",20220518_1348_Plate17,6_20220509,2022-05-18 13:48:00
17,/scratch-shared/amftrack/stitch_temp/20220510_...,6,17,'Carrot','Agg','001P100N100C','Y','-',20220509,25,...,False,False,False,False,False,False,"10.05.2022, 18:51:",20220510_1851_Plate17,6_20220509,2022-05-10 18:51:00


In [8]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders.loc[folders['/Img/TileConfiguration.txt.registered']==True]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
3,/scratch-shared/amftrack/stitch_temp/20220407_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"07.04.2022, 21:55:",20220407_2155_Plate33,889_20220325,2022-04-07 21:55:00
18,/scratch-shared/amftrack/stitch_temp/20220327_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"27.03.2022, 15:35:",20220327_1535_Plate14,889_20220325,2022-03-27 15:35:00
30,/scratch-shared/amftrack/stitch_temp/20220330_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"30.03.2022, 21:40:",20220330_2140_Plate14,889_20220325,2022-03-30 21:40:00
39,/scratch-shared/amftrack/stitch_temp/20220419_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"19.04.2022, 01:20:",20220419_0120_Plate33,889_20220325,2022-04-19 01:20:00
62,/scratch-shared/amftrack/stitch_temp/20220328_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"28.03.2022, 05:36:",20220328_0536_Plate14,889_20220325,2022-03-28 05:36:00
64,/scratch-shared/amftrack/stitch_temp/20220415_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"15.04.2022, 05:55:",20220415_0555_Plate33,889_20220325,2022-04-15 05:55:00
86,/scratch-shared/amftrack/stitch_temp/20220401_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"01.04.2022, 18:45:",20220401_1845_Plate14,889_20220325,2022-04-01 18:45:00
98,/scratch-shared/amftrack/stitch_temp/20220330_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"30.03.2022, 23:40:",20220330_2340_Plate14,889_20220325,2022-03-30 23:40:00
101,/scratch-shared/amftrack/stitch_temp/20220329_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"29.03.2022, 21:10:",20220329_2110_Plate14,889_20220325,2022-03-29 21:10:00
112,/scratch-shared/amftrack/stitch_temp/20220418_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"18.04.2022, 01:20:",20220418_0120_Plate33,889_20220325,2022-04-18 01:20:00


In [12]:
from amftrack.pipeline.launching.run_super import run_parallel,run_launcher

for plate in plates:
    select = folders.loc[folders["Plate"] == plate]
    num_parallel = 128
    time = "12:00:00"
    thresh = 10000  # For R. irregularis, thresh 10000 is good. For Aggregatum, higher may be necessary
    args = [thresh, directory_targ]
    run_parallel(
        "final_alignment.py",
        args,
        select,
        num_parallel,
        time,
        "realign",
        cpus=128,
        node="fat",
        name_job = name_job

    )

Submitted batch job 1138954


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [20]:
plates

['6_20220509']

In [19]:
select

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime


In [13]:
i= 0 
run_info = folders
folder_list = list(run_info["folder"])
folder_list.sort()

dates_datetime_chosen = folder_list[i : i + 2]
print("========")
print(f"Matching plate {dates_datetime_chosen[0]} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen

Matching plate 20220510_1851_Plate17 at dates ['20220510_1851_Plate17', '20220511_1750_Plate17']


In [17]:
from pymatreader import read_mat

from amftrack.pipeline.functions.image_processing.extract_graph import (
    sparse_to_doc,
    get_degree3_nodes,
)
import open3d as o3d
from cycpd import rigid_registration
import pandas as pd
import os
dilateds = []
skels = []
skel_docs = []
for directory_name in dates:
    path_snap = os.path.join(directory_targ, directory_name)
    skel_info = read_mat(path_snap + "/Analysis/skeleton_pruned.mat")
    skel = skel_info["skeleton"]
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)

In [18]:
skel_docs

[{(9706, 41696): 1,
  (9706, 41697): 1,
  (9706, 41698): 1,
  (9707, 41694): 1,
  (9707, 41695): 1,
  (9708, 41692): 1,
  (9708, 41693): 1,
  (9708, 41694): 1,
  (9709, 41690): 1,
  (9709, 41691): 1,
  (9709, 41692): 1,
  (9710, 41690): 1,
  (9711, 41688): 1,
  (9711, 41689): 1,
  (9712, 41685): 1,
  (9712, 41686): 1,
  (9712, 41687): 1,
  (9713, 41682): 1,
  (9713, 41683): 1,
  (9713, 41684): 1,
  (9714, 41679): 1,
  (9714, 41680): 1,
  (9714, 41681): 1,
  (9715, 41677): 1,
  (9715, 41678): 1,
  (9716, 41675): 1,
  (9716, 41676): 1,
  (9716, 41677): 1,
  (9717, 41674): 1,
  (9717, 41675): 1,
  (9718, 41673): 1,
  (9718, 41674): 1,
  (9719, 41672): 1,
  (9720, 41670): 1,
  (9720, 41671): 1,
  (9721, 41666): 1,
  (9721, 41667): 1,
  (9721, 41668): 1,
  (9721, 41669): 1,
  (9722, 41663): 1,
  (9722, 41664): 1,
  (9722, 41665): 1,
  (9723, 41660): 1,
  (9723, 41661): 1,
  (9723, 41662): 1,
  (9724, 41658): 1,
  (9724, 41659): 1,
  (9725, 41655): 1,
  (9725, 41656): 1,
  (9725, 41657): 1,
